<h3>Načitanie knižníc</h3>

In [1]:
import numpy as np
import random

<h3>Definovanie premenných</h3>

In [2]:
# Parametre portfólia
NUM_STOCKS = 10  # Počet akcií
POPULATION_SIZE = 100  # Veľkosť populácie
GENERATIONS = 10  # Počet generácií
MUTATION_RATE = 0.1  # Miera mutácie

# Generovanie náhodných ročných výnosov a rizík (volatilita)
np.random.seed(42)
expected_returns = np.random.uniform(0.05, 0.50, NUM_STOCKS)  # Očakávané ročné výnosy
risks = np.random.uniform(0.1, 0.5, NUM_STOCKS)  # Očakávané riziká (volatilita)

<h3>Definovanie fitness funkcie</h3>

In [3]:
def fitness_function(weights):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(np.diag(risks**2), weights)))
    return portfolio_return - portfolio_risk

<h3>Normalizovanie váh</h3>

In [4]:
def normalize_weights(weights):
    return weights / np.sum(weights)

<h3>Inicializácia populácie</h3>

In [5]:
def generate_population(size):
    return [normalize_weights(np.random.rand(NUM_STOCKS)) for _ in range(size)]

<h3>Metódy selekcie</h3>

In [6]:
# Turnajová selekcia
def tournament_selection(population):
    tournament = random.sample(population, 5)
    return max(tournament, key=fitness_function)

# Ruletový výber
def roulette_selection(population):
    fitness_values = np.array([fitness_function(ind) for ind in population])
    fitness_sum = np.sum(fitness_values)
    selection_probs = fitness_values / fitness_sum
    return population[np.random.choice(len(population), p=selection_probs)]

# Výber podľa poradia (Rank Selection)
def rank_selection(population):
    fitness_values = np.array([fitness_function(ind) for ind in population])
    ranked_indices = np.argsort(-fitness_values)
    selection_probs = 1 / (np.arange(len(population)) + 1)
    selection_probs /= np.sum(selection_probs)
    return population[np.random.choice(len(population), p=selection_probs)]

<h3> Funkcie mutácie a kríženia</h3>

In [7]:
# Kríženie
def crossover(parent1, parent2):
    point = random.randint(1, NUM_STOCKS - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return normalize_weights(child1), normalize_weights(child2)

# Mutácia
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx = random.randint(0, NUM_STOCKS - 1)
        individual[idx] = random.uniform(0.0, 1.0)
        return normalize_weights(individual)
    return individual

<h3>Implementácia genetického algoritmu</h3>

In [8]:
def genetic_algorithm(selection_method):
    population = generate_population(POPULATION_SIZE)
    
    for generation in range(GENERATIONS):
        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1 = selection_method(population)
            parent2 = selection_method(population)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))
        
        population = new_population
        
        best_solution = max(population, key=fitness_function)
        yield generation, best_solution, fitness_function(best_solution)

<h3>Porovnávanie metód selekcie</h3>

In [9]:
def compare_selections():
    methods = {
        'Turnajová selekcia': tournament_selection,
        'Ruletový výber': roulette_selection,
        'Výber podľa poradia': rank_selection
    }

    results = {}
    
    for method_name, method in methods.items():
        print(f"\n{method_name}:\n" + "=" * 30)
        best_portfolio = None
        best_fitness = float('-inf')
        for generation, solution, fitness in genetic_algorithm(method):
            print(f"Generácia {generation + 1}: Najlepšie váhy: {solution}, Výnos: {fitness:.4f}")
            if fitness > best_fitness:
                best_fitness = fitness
                best_portfolio = solution
        
        results[method_name] = (best_portfolio, best_fitness)

    print("\nNajlepšie výsledky:")
    for method, (portfolio, fitness) in results.items():
        print(f"{method}: Výnos: {fitness:.4f}, Váhy: {portfolio}")

<h3>Spustenie kódu</h3>

In [10]:
compare_selections()


Turnajová selekcia:
Generácia 1: Najlepšie váhy: [0.04982109 0.10074521 0.11842311 0.15084054 0.03752577 0.00199896
 0.02488473 0.16361632 0.15886635 0.19327793], Výnos: 0.2428
Generácia 2: Najlepšie váhy: [0.05782679 0.11884556 0.0947377  0.28363317 0.01603616 0.00935068
 0.01551684 0.20041191 0.02959902 0.17404216], Výnos: 0.2471
Generácia 3: Najlepšie váhy: [0.05080487 0.10441412 0.08323368 0.24919153 0.01408889 0.00821522
 0.01363263 0.17607585 0.13549682 0.1648464 ], Výnos: 0.2494
Generácia 4: Najlepšie váhy: [0.04995261 0.10266255 0.08183742 0.24501128 0.01385254 0.00807741
 0.01340394 0.17312213 0.14079181 0.17128832], Výnos: 0.2497
Generácia 5: Najlepšie váhy: [0.01314795 0.11413125 0.09097969 0.27238214 0.01540005 0.00897976
 0.01490133 0.19246207 0.10118638 0.17642939], Výnos: 0.2511
Generácia 6: Najlepšie váhy: [0.01407736 0.10949873 0.08728688 0.26132631 0.01477497 0.00861528
 0.01429649 0.18465014 0.12268319 0.18279064], Výnos: 0.2515
Generácia 7: Najlepšie váhy: [0.01238